In [1]:
%matplotlib widget
import pandas as pd
import numpy as np

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

# local import
import lppydsmc as ld

In [2]:
dir_path = Path('../../../lppydsmc/simulations/results').resolve()

In [46]:
name_simulation = 'thruster_two_grids_ions' # 'thruster_two_grids_neutrals' # thruster_two_grids thruster_three_grids_neutrals thruster_three_grids

In [47]:
store = pd.HDFStore(dir_path/Path(name_simulation)/'monitoring.h5')
print(store.keys())

['/fluxes', '/out_particles', '/particles', '/wall_collisions']


# Parameters of the simulation

If you used the option `-s` (or `save`) when launching the simulation, it can be found under `<dir_path>/<name_simulation>/params.ini`.

**ADAPT THE FOLLOWING VALUES TO YOUR SIMULATION PARAMETERS.**

In [48]:
time_step = 1e-8
steps_between_savings = 1
dsmc_grid_shape = np.array([16,5], dtype = int)
offsets = np.array([0,0])
system_size = np.array([0.016,0.005])
dx = dy = 1e-3
dz = 0.001
cell_volume = dx*dy*dz
particles_weight = 320000

# Analysis of the results

First, we will simply focus on getting the density and velocity functions out to launch on background gas a better simulation.

In [49]:
df_particles = store['particles']

In [50]:
df_particles['v'] = np.sqrt(df_particles['vx']**2+df_particles['vy']**2+df_particles['vz']**2)

In [51]:
fig, ax = plt.subplots(2,2,constrained_layout = True)
df = df_particles[(df_particles['x']>10e-3) & (df_particles.index * time_step > 4e-5)]
sns.histplot(ax = ax[0,0], data = df, x = 'vx', color = 'r')
sns.histplot(ax = ax[0,1], data = df, x = 'vy', color = 'g')
sns.histplot(ax = ax[1,0], data = df, x = 'vz', color = 'b')
sns.histplot(ax = ax[1,1], data = df, x = 'v', color = 'k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='v', ylabel='Count'>

In [52]:
evo_nb_particles = df_particles['species'].groupby(df_particles.index).count()
print(evo_nb_particles)

1          7
2         14
3         21
4         28
5         35
        ... 
9996     981
9997     983
9998     987
9999     988
10000    987
Name: species, Length: 10000, dtype: int64


In [53]:
fig, ax = plt.subplots(constrained_layout = True)
ax.plot(evo_nb_particles.index*time_step, evo_nb_particles)
ax.set_xlabel('time (s)')
ax.set_ylabel('Number of particles');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The previous figure shows at what point we are arriving in a more steady state.

In [54]:
steady_state = int(2e-5/time_step)
# df_particles_steady = df_particles[df_particles.index > steady_state]

At this point, the goal is simply to create *density* and *dynamic* functions from the results in steady flow. 
To start with, we will select a mesh (in practice we use the same as for the DSMC), compute the values and interpolate them in a function that we save for later use.

In [11]:
interpolate = True
if(interpolate):
    X, Y, density_arr, dynamic_arr = ld.background_gas.interpolate.interpolate(df_particles, offsets, dsmc_grid_shape, system_size, cell_volume, particles_weight, steady_state = steady_state)
    np.save('gas_density_arr_{}.npy'.format(name_simulation), density_arr, allow_pickle = True)
    np.save('gas_dynamic_arr_{}.npy'.format(name_simulation), dynamic_arr, allow_pickle = True)
    np.save('x_arr_{}.npy'.format(name_simulation), X, allow_pickle = True)
    np.save('y_arr_{}.npy'.format(name_simulation), Y, allow_pickle = True)
    # ld.background_gas.interpolate.read_interpolation()

In [12]:
# checking the value to make sure we don't have anything unwanted

# TODO: the function is inverted !!!!!!! (should invert the axes I believe - Or I did something wrong ... I don't know)
path_density_arr = 'gas_density_arr_{}.npy'.format(name_simulation) 
path_dynamic_arr = 'gas_dynamic_arr_{}.npy'.format(name_simulation)
path_x = 'x_arr_{}.npy'.format(name_simulation)
path_y = 'y_arr_{}.npy'.format(name_simulation)
density_fn, dynamic_fn = ld.background_gas.interpolate.read_interpolation(path_x, path_y, path_density_arr, path_dynamic_arr)

In [16]:
X, Y = np.linspace(0,16e-3,100), np.linspace(0,5e-3,25)
Z_dynamic = np.zeros((Y.shape[0],X.shape[0],3)) # this one is random ! (and this is not strictly speaking the one we are interested into)
Z_density = np.zeros((Y.shape[0],X.shape[0]))
for i, x in enumerate(X):
    for j, y in enumerate(Y):
        Z_dynamic[j,i,:] = dynamic_fn(np.array([x]),np.array([y]))[0]
        Z_density[j,i] = density_fn(np.array([x]),np.array([y]))[0]

In [17]:
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

# Make data.
X, Y = np.meshgrid(X, Y)
vmax = np.max(Z_density)
vmin = np.min(Z_density)
# Plot the surface.
midpoint = 1 - vmax / (vmax + abs(vmin))
orig_cmap = cm.coolwarm
shifted_cmap = shiftedColorMap(orig_cmap, midpoint=midpoint, name='shifted')

surf = ax.plot_surface(X, Y, Z_density, cmap=shifted_cmap,
                       linewidth=0, antialiased=False)

# Customize the z axis.
# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap


In [42]:
np.where(Z_density<0)

(array([32, 32, 32, ..., 99, 99, 99]), array([ 6,  7,  8, ..., 91, 92, 93]))

## Check output thruster

In [55]:
df_out = store['out_particles'] # df particles should be updated at every frame ... but anyway

In [56]:
df_out = df_out[df_out['x']>1e-3]

In [57]:
df_out.head()

,x,y,vx,vy,vz,species
129,0.016007,0.002651,20966.043247,2225.182062,403.717186,0.0
130,0.016110,0.002324,20959.543433,-2334.380589,-380.188825,0.0
130,0.016159,0.002122,21091.500830,1014.261344,322.332924,0.0
130,0.016166,0.002832,21074.841685,1000.237913,-323.362922,1.0
130,0.016022,0.002661,21072.378862,431.880745,434.604799,1.0


In [58]:
df_out_neutrals = df_out[df_out['species'] == 0.0]
df_out_ions = df_out[df_out['species'] == 1.0]

In [59]:
groups_neutrals = df_out_neutrals.groupby(df_out_neutrals.index).count()
groups_ions = df_out_ions.groupby(df_out_ions.index).count()
fig, ax = plt.subplots(constrained_layout = True)
bins = 20
factor_neutrals =  bins/groups_neutrals.shape[0]
factor_ions = bins/groups_ions.shape[0]
ax.hist(groups_neutrals.index*time_step, weights = factor_neutrals*groups_neutrals['species'], bins = bins, histtype = 'step')
ax.hist(groups_ions.index*time_step, weights = factor_ions*groups_ions['species'], bins = bins, histtype = 'step');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
df_out.groupby('species').count()

,x,y,vx,vy,vz
species,,,,,
0.0,42442,42442,42442,42442,42442
1.0,7573,7573,7573,7573,7573


In [62]:
1-7573/42442

0.8215682578577824

In [61]:
1-7046/42382

0.8337501769619178

In [63]:
df_out.groupby('species').mean()

,x,y,vx,vy,vz
species,,,,,
0.0,0.016102,0.002500,20217.771394,22.552055,0.531562
1.0,0.016108,0.002498,21078.482261,-4.985717,-9.193084


In [64]:
fig, ax = plt.subplots()
sns.histplot(data = df_out[df_out['vx']>18e3], x = 'vx', hue = 'species', bins = 100)
# ax.set_yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='vx', ylabel='Count'>

## Particles collisions with walls

In [65]:
df_wall_collisions = store['wall_collisions']

In [66]:
# fig, ax = plt.subplots(constrained_layout = True)
sns.jointplot(data = df_wall_collisions, x = 'x', y = 'y', hue = 'species', s = 0.3)
# plt.legend(loc = 'lower center') 
# ax=ax, 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …